<a href="https://colab.research.google.com/github/theMADAIguy/STOCK-PREDICTION-USING-SVM-REGRESSION/blob/master/text_simple_rnn_tutorial_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchtext
from torchtext import data
from torchtext import datasets
import random
import torch.nn as nn
import torch.optim as optim

#Constants
SEED = 1234
MAX_SIZE_VOCAB=25000
BATCH_SIZE=64
EPOCHS=10


torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/
PATH = './text_rnn_model.pt'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [3]:
TEXT=data.Field(tokenize='spacy')
LABEL=data.LabelField(dtype=torch.float)

trainSet, testSet = datasets.IMDB.splits(TEXT, LABEL)
testSet, validSet = testSet.split(random_state=random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.3MB/s]


In [0]:
TEXT.build_vocab(trainSet, max_size= MAX_SIZE_VOCAB)
LABEL.build_vocab(trainSet)

In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((trainSet, validSet, testSet), batch_size=BATCH_SIZE, device=device)

In [0]:
class RNN(nn.Module):

  def __init__(self, inp_dim, embed_dim, hidden_dim, out_dim):
    super().__init__()
    self.embedding=nn.Embedding(inp_dim, embed_dim)
    self.rnn=nn.RNN(embed_dim, hidden_dim)
    self.fc=nn.Linear(hidden_dim, out_dim)

  def forward(self, text):

    embedded=self.embedding(text)
    output, hidden =self.rnn(embedded)
    fc=self.fc(hidden.squeeze(0))
    return fc
  

In [7]:
INPUT_DIMENSION=len(TEXT.vocab)
EMBEDDING_DIMENSION=100
HIDDEN_DIMENSION=256
OUTPUT_DIMENSION=1

model=RNN(INPUT_DIMENSION,EMBEDDING_DIMENSION, HIDDEN_DIMENSION, OUTPUT_DIMENSION)
print(model)

RNN(
  (embedding): Embedding(25002, 100)
  (rnn): RNN(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


In [0]:
optimizer=optim.SGD(model.parameters(), lr=0.001)
criterion=nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def accuracy(predicted, labels):

  rounded=torch.round(torch.sigmoid(predicted))
  correct=(rounded==labels).float()
  acc=correct.sum()/len(rounded)
  return acc

In [0]:
def train(model, iterator, optimizer, criterion):

  epoch_loss=0.0
  epoch_acc=0.0

  model.train()

  for batch in iterator:
    model.zero_grad()
    output=model(batch.text).squeeze(1)
    loss=criterion(output, batch.label)
    acc=accuracy(output, batch.label)
    loss.backward()
    optimizer.step()

    epoch_loss+=loss.item()
    epoch_acc+=acc.item()

  return epoch_loss/len(iterator), epoch_acc / len(iterator)


In [0]:
def evaluate(model, iterator, optimizer, criterion):

  epoch_loss=0.0
  epoch_acc=0.0

  model.eval()

  with torch.no_grad():

    for batch in iterator:
      output=model(batch.text).squeeze(1)
      loss=criterion(output, batch.label)
      acc=accuracy(output, batch.label)
      
      epoch_loss+=loss.item()
      epoch_acc+=acc.item()

  return epoch_loss/len(iterator), epoch_acc / len(iterator)




In [13]:
best_valid_loss=float('inf')
for epoch in range(EPOCHS):

  train_loss, train_acc=train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc=evaluate(model, valid_iterator, optimizer, criterion)

  if(valid_loss < best_valid_loss):
    best_valid_loss=valid_loss
    torch.save(model.state_dict(), PATH)
  
  print("Epoch: ", epoch, "train_loss: ", train_loss, "valid_loss: ", valid_loss, "train_acc: ", train_acc, "valid_acc: ", valid_acc)

Epoch:  0 train_loss:  0.693708213851275 valid_loss:  0.6943505911503808 train_acc:  0.5042279411459822 valid_acc:  0.511917372881356
Epoch:  1 train_loss:  0.6933424660311941 valid_loss:  0.6942130448454518 train_acc:  0.4929108057180634 valid_acc:  0.5131091101694916
Epoch:  2 train_loss:  0.6933092689880019 valid_loss:  0.6941131403890707 train_acc:  0.4920955882657824 valid_acc:  0.512447033898305
Epoch:  3 train_loss:  0.6932683250178462 valid_loss:  0.6943359991251412 train_acc:  0.5018781969309463 valid_acc:  0.5127118644067796
Epoch:  4 train_loss:  0.6933856772644745 valid_loss:  0.6941945552825928 train_acc:  0.4923833120052162 valid_acc:  0.5136387711864406
Epoch:  5 train_loss:  0.6933741683850203 valid_loss:  0.693930423865884 train_acc:  0.49409367010721467 valid_acc:  0.5107697743480488
Epoch:  6 train_loss:  0.6930962525060415 valid_loss:  0.6936140307935618 train_acc:  0.5035805627513115 valid_acc:  0.5155367235005912
Epoch:  7 train_loss:  0.6933590765194515 valid_los

In [15]:
model.load_state_dict(torch.load(PATH))

test_loss, test_acc=evaluate(model, test_iterator, optimizer, criterion)
print("test_loss: ", test_loss, "test_acc: ", test_acc*100)

test_loss:  0.6987875858797644 test_acc:  49.708355060459056
